--- ## Draft of a presentation - include it in Makefile if you run it apart paper.qmd too
title: |
  | Earnings Management and Investor Protection:
  | Accounting Reading Group
author: "Mel Mzv"
institute: "Humboldt-Universität zu Berlin – Summer 2025"
date: today
fontsize: "9pt"
pdf-engine: xelatex # pdflatex creates rastered fonts
format: 
  beamer:
    slide-level: 3
    number-sections: true
    toc: false
header-includes:
- \usepackage{booktabs} 
- \usepackage{threeparttable}
- \usepackage{graphicx}
- \usepackage{etoolbox}
- \AtBeginEnvironment{verbatim}{\fontsize{7}{7}\selectfont}  # This sets the font size for verbatim environments
- \input{beamer_theme_trr266.sty}
bibliography: references.bib
biblio-style: apsr

---

In [ ]:
#| echo: false
#| output: false

import pickle
import pandas as pd

# Load the results from the existing pickle file for table_1
with open('../data/generated/table_1.pickle', 'rb') as f:
    results = pickle.load(f)

# Load the results from the new pickle file for final_table and summary_stats
with open('../output/em_results.pickle', 'rb') as f:
    em_results = pickle.load(f)

def escape_special_characters(df):
    df.columns = [col.replace('_', '\\_').replace('#', '\\#') for col in df.columns]
    return df

# Custom formatter function
def custom_float_format(x):
    return f"{x:.3f}".lstrip('0') if pd.notnull(x) else ""

def prep_latex_table(df, caption=None, label=None, fontsize='\\fontsize{11}{13}\\selectfont', description=None):
    df = escape_special_characters(df.reset_index(drop=True))  # Drop the index column from output tables for better readability
    num_columns = len(df.columns)
    column_format = 'l' + 'r' * (num_columns - 1)
    
    latex_table = df.to_latex(
        column_format=column_format,
        index=False, 
        float_format=custom_float_format,
        escape=False  # Important to disable escaping so LaTeX commands in column names are respected
    )
    
    # Construct description to the table
    full_caption = f"{caption}\\\\\n\\textnormal{{{description}}}" if description else caption
    
    latex_table_lines = [
        "\\begin{table}[htbp]",
        "\\centering",
        fontsize,  # Set font size for the table
        latex_table,
        f"\\caption{{{full_caption}}}" if full_caption else "",
        f"\\label{{{label}}}" if label else "",
        "\\end{table}"
    ]
    return "\n".join(line for line in latex_table_lines if line)

# Motivation

## Understanding Systematic Differences in Earnings Management across Countries

- The paper of @Leuz_2003 focuses on the relationship between investor protection and earnings management.
- Investor protection plays a crucial role in limiting the extent of earnings management by reducing the opportunities for managers to manipulate earnings.
- Analyzing the relationship between EM and investor protection across different countries provides insights into how legal and institutional frameworks impact financial reporting practices.
- Earnings management (EM) measures reflect the extent to which managers manipulate financial reports to meet certain objectives, such as meeting earnings targets or influencing stock prices.

## Project Objective

- The primary objective is to demonstrate a reproducible and collaborative research workflow by calculating and analyzing EM measures across different countries.
- The study aims to replicate and extend the findings of @Leuz_2003, exploring the relationship between EM and investor protection using Worldscope datasets for the time range 1990 to 1999.
- The analysis provides insights into the levels of earnings management across countries, comparing them with the original findings of Leuz, Nanda, and Wysocki (2003).

## Project Relevance

- Understanding EM trends across countries aids in assessing global financial transparency.
- Promotes transparency and reproducibility in empirical accounting research.

# Research Design

- Data sourced from the Worldscope Database (1990-1999) July 2024 version.
- Replication includes calculation of **four earnings management measures**:
  1. Earnings smoothing using accruals (EM1)
  2. Correlation between changes in accruals and cash flows (EM2)
  3. Magnitude of accruals (EM3)
  4. Ratio of small profits to small losses (EM4)

# Assumptions

- **Database Version**: Analysis uses the July 2024 Worldscope data, which may differ from the November 2000 version due to updates and restatements.
- **Key Terms**: Earnings management (EM) and investor protection defined as per @Leuz_2003 for consistency.
- **Handling Negative Values**: Negative values in key metrics (e.g., operating income) were included to capture the full scope of EM activities.
- **Variable Selection**: Chose specific Worldscope variables (e.g., item1151 over item4051) based on best practices; differences may affect results.
- **Currency Consistency**: EM measures are based on scaled variables; currency consistency ensures comparability across countries.

# Data - Replication Steps 

Step 1: Pulling the Data and Managing the Databases

Step 2: Data Preparation

+  To verify the robustness of descriptive statistics before the main replication, Table 1 from Leuz et al. was replicated, yielding results that closely align with the original study:
---

In [ ]:
#| label: table_1
#| echo: false
#| output: asis

print(prep_latex_table(
    results['table_1'],
    caption='Replicated table - Descriptive statistics: Number of firm-year observations per country',
    label='tab:table_1',
    fontsize='\\fontsize{5pt}{6pt}\\selectfont'  # Set smaller font size
))

---

Step 3: Analysis Implementation and Table Reproduction

- Smoothing Reported Operating Earnings Using Accruals (EM1)
    $$
    \text{EM1} = \text{Median}\left(\frac{\frac{\sigma(\text{Operating Income}_{it})}{\text{Total Assets}_{i(t-1)}}}{\frac{\sigma(\text{Operating Cash Flow}_{it})}{\text{Total Assets}_{i(t-1)}}}\right)
    $$
- Smoothing and the Correlation Between Changes in Accounting Accruals and Operating Cash Flows (EM2)
    $$
    \text{EM2} = \rho\left(\frac{\Delta \text{Accruals}_{it}}{\text{Total Assets}_{i(t-1)}}, \frac{\Delta \text{CFO}_{it}}{\text{Total Assets}_{i(t-1)}}\right)
    $$
- Discretion in Reported Earnings: The Magnitude of Accruals (EM3)
    $$
    \text{EM3} = \text{Median}\left(\frac{|\text{Accruals}_{it}|}{|\text{CFO}_{it}|}\right)
    $$

 ---
- Discretion in Reported Earnings: Small Loss Avoidance (EM4)
    $$
    \text{EM4} = \frac{\text{Small Profits}}{\text{Small Losses}}
    $$
 - Aggregate Measure of Earnings Management
    $$
    \text{Aggregate EM Score}_{\text{country}} = \frac{\text{Rank}_{\text{EM1}} + \text{Rank}_{\text{EM2}} + \text{Rank}_{\text{EM3}} + \text{Rank}_{\text{EM4}}}{4}
    $$

# Results

In [ ]:
#| label: final_combined_table
#| echo: false
#| output: asis

# Rename the columns to ensure consistency with paper
em_results['final_combined_table'].rename(columns={
    'item6026': 'Country',
    'EM1': 'EM1 (−)',
    'EM2': 'EM2 (−)',
    'EM3': 'EM3 (+)',
    'EM4': 'EM4 (+)',
    'Aggregate_EM_Score': 'Aggregate EM score'
}, inplace=True)

print(prep_latex_table(
    em_results['final_combined_table'], 
    caption='Replicated table - Country scores for earnings management measures (Sorted by aggregate earnings management)',
    label='tab:final_combined_table',
    fontsize='\\fontsize{4pt}{5pt}\\selectfont',  # Set smaller font size

))

# Conclusion

## Summary of Findings
- EM1 Alignment: Strong alignment with minor differences, reflecting consistent trends in earnings management across countries.
- EM2 Findings: Similar patterns observed; Greece and Japan show more smoothing, while Canada and the U.S. show less.
- EM3 Consistency: Replicated results closely match the original, confirming trends in accrual magnitudes.
- EM4 Regional Differences: Significant regional variations in loss avoidance, consistent with original findings.
- Aggregate Scores: Strong alignment between original and replicated scores, confirming key trends across countries.
- **Overall Conclusion**: Replication confirms higher EM in European and Asian countries compared to Anglo-American counterparts.

## Key Insights

- Significant differences in earnings management practices across countries.
- Strong alignment with original findings in Leuz, Nanda, and Wysocki (2003).
- The project structure demonstrated the effectiveness of an open science and collaborative workflow.
- **Future Work**: Repository can be cloned or forked for further analysis.

# References {-}
\setlength{\parindent}{-0.2in}
\setlength{\leftskip}{0.2in}
\setlength{\parskip}{8pt}
\noindent
